In [6]:
import xmltodict
import xlwt
import traceback
import numpy as np
import pandas as pd
from time import gmtime, strftime
import argparse
import json
import re
from functools import reduce
import os
import glob
import base64
import struct
import matplotlib.pyplot as plt

In [7]:
pwd

'/Users/pae2/OneDrive - cumc.columbia.edu/CADnet/DummyData'

In [11]:

# fpath_train =
# fpath_eval ="/labs/cardiac-amyloid/ValveNet/data/aortic_stenosis_concat_eval.npy"
npyfilespath ='/Users/pae2/OneDrive - cumc.columbia.edu/CADnet/DummyData/'
os.chdir(npyfilespath)
npfiles= glob.glob( "*.npy")
npfiles.sort()
all_arrays_train = []
# all_arrays_eval = []

#If trying to test model quickly use smaller total dataset or change dataloader to load npy file batch by batch

#Not rewriting stacked array so below is commented out

for i, npfile in enumerate(npfiles):
    x = 0
    i = 0
    try:
        path = os.path.join(npyfilespath + npfile)
        file=np.load(path)

        file=np.reshape(file,(1,2500,12))
        all_arrays_train.append(file)
        x += 1
        i += 1
    except:
        continue
    if i % 1 == 100:
        print("{i} EKGs have been written to array")

# for i, npfile in enumerate(filelist_eval):
#     x = 0
#     i = 0
#     try:
#         path = os.path.join(npyfilespath + npfile)
#         file=np.load(path)
#         file=np.reshape(file,(1,2500,12))
#         all_arrays_eval.append(file)
#         x += 1
#         i += 1
#     except:
#         continue
#     if i % 1 == 100:
#         print("{i} EKGs have been written to array")

# # for i, npfile in enumerate(filelist_train):
# #     x = 0
# #     i = 0
# #     start = time()
# #     try:
# #         all_arrays_train[i]=np.load(npyfilespath + npfile)
# #         x += 1
# #         i += 1
# #         # all_arrays_train.append(np.load(npyfilespath + npfile))
# #     except:
# #         i += 1
# #     if i % 1 == 0:
# #         print(time() - start, time()-start_loop)


# print('train array count:' ,len(all_arrays_train), '\n eval array count:', len(all_arrays_eval))


In [14]:
np.shape(all_arrays_train)

(33, 1, 2500, 12)

In [2]:
path = './MUSE_20200923_093504_79000_example.xml'

In [3]:
with open(path, 'rb') as fd:
    dic = xmltodict.parse(fd.read().decode('utf8'))

In [4]:
dic

OrderedDict([('RestingECG',
              OrderedDict([('MuseInfo',
                            OrderedDict([('MuseVersion', '9.0.10.18530')])),
                           ('PatientDemographics',
                            OrderedDict([('PatientID', '201700237'),
                                         ('PatientAge', '40'),
                                         ('AgeUnits', 'YEARS'),
                                         ('DateofBirth', '06-10-1980'),
                                         ('Gender', 'MALE'),
                                         ('Race', 'ASIAN'),
                                         ('HeightCM', '180'),
                                         ('WeightKG', '93'),
                                         ('PatientLastName', 'MUSE'),
                                         ('PatientFirstName', 'TESTONE')])),
                           ('TestDemographics',
                            OrderedDict([('DataType', 'RESTING'),
                               

In [5]:
dic['RestingECG']['PatientDemographics']

OrderedDict([('PatientID', '201700237'),
             ('PatientAge', '40'),
             ('AgeUnits', 'YEARS'),
             ('DateofBirth', '06-10-1980'),
             ('Gender', 'MALE'),
             ('Race', 'ASIAN'),
             ('HeightCM', '180'),
             ('WeightKG', '93'),
             ('PatientLastName', 'MUSE'),
             ('PatientFirstName', 'TESTONE')])

In [6]:
dic['RestingECG']['PharmaData']['PharmaUniqueECGID']

'SCD11041674PA24092019090332'

In [7]:
def decode_ekg_muse(raw_wave):
    """
    Ingest the base64 encoded waveforms and transform to numeric
    """
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)

    return byte_array


In [8]:
lead_data = dict()
for lead in dic['RestingECG']['Waveform']:
    for leadid in range(len(lead['LeadData'])):
        lead_data[lead['LeadData'][leadid]['LeadID']] = decode_ekg_muse(lead['LeadData'][leadid]['WaveFormData'])

In [9]:
lead_data['III'] = list(np.array(lead_data["II"]) - np.array(lead_data["I"]))
lead_data['aVR'] = list(-(np.array(lead_data["I"]) + np.array(lead_data["II"]))/2)
lead_data['aVF'] = list((np.array(lead_data["II"]) - np.array(lead_data["I"]))/2)
lead_data['aVL'] = list((np.array(lead_data["I"]) - np.array(lead_data["II"]))/2)

In [10]:
list(lead_data.keys())

['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'III', 'aVR', 'aVF', 'aVL']

In [11]:
lead_keys = list(lead_data.keys())

In [12]:
lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']


In [13]:
dic['RestingECG']['Waveform']

[OrderedDict([('WaveformType', 'Median'),
              ('WaveformStartTime', '0'),
              ('NumberofLeads', '8'),
              ('SampleType', 'CONTINUOUS_SAMPLES'),
              ('SampleBase', '500'),
              ('SampleExponent', '0'),
              ('HighPassFilter', '16'),
              ('LowPassFilter', '150'),
              ('ACFilter', 'NONE'),
              ('LeadData',
               [OrderedDict([('LeadByteCountTotal', '1200'),
                             ('LeadTimeOffset', '0'),
                             ('LeadSampleCountTotal', '600'),
                             ('LeadAmplitudeUnitsPerBit', '4.88'),
                             ('LeadAmplitudeUnits', 'MICROVOLTS'),
                             ('LeadHighLimit', '32767'),
                             ('LeadLowLimit', '-32768'),
                             ('LeadID', 'I'),
                             ('LeadOffsetFirstSample', '0'),
                             ('FirstSampleBaseline', '0'),
                

In [14]:

#this is muse_xml_to_array.py
#Input a directory of XML files, get a directory of np arrays where each .npy is a 12-lead ecg shape 2500,12,1. So this gives you JUST the waveforms
#Some notes, the unique ECG ID index key used in MUSE backend does not exist in the XML at least for us, so instead we use MRN_AcquisitionDTTM_PharmaUniqueECGID

#In terminal run python3 muse_xml_to_array.py <LOCATION_OF_XML_FILES>

import pandas as pd 
import numpy as np 
import xmltodict
import base64
import struct
import argparse
import os
import sys



def file_path(path):
    filepath = path
    for dirName, subdirList, fileList in os.walk(filepath):
        for filename in fileList:
            if ".xml" in filename.lower():
                ekg_file_list.append(os.path.join(dirName, filename))
    
#need to update this function to check the output directory for the output file and then only on newly added EKGs
#add timestamp to start file string 
#this is annoying because the XML file name is a random timestamp and the output file is the UniqueECGID


if not os.path.exists(os.getcwd() + '/ekg_waveforms_output/'):
    os.mkdir(os.getcwd() + '/ekg_waveforms_output/')

# parser = argparse.ArgumentParser(description='Input and outputs for XML EKG parsing')
# parser.add_argument('input', type=str)
# parser.set_defaults(output=os.getcwd() + '/ekg_waveforms_output/') #ensure this directory already exists

# args = parser.parse_args()



def decode_ekg_muse(raw_wave):
    """
    Ingest the base64 encoded waveforms and transform to numeric
    """
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)
    return byte_array


def decode_ekg_muse_to_array(raw_wave, downsample = 1):
    """
    Ingest the base64 encoded waveforms and transform to numeric

    downsample: 0.5 takes every other value in the array. Muse samples at 500/s and the sample model requires 250/s. So take every other.
    """
    try:
        dwnsmpl = int(1//downsample)
    except ZeroDivisionError:
        print("You must downsample by more than 0")
    # covert the waveform from base64 to byte array
    arr = base64.b64decode(bytes(raw_wave, 'utf-8'))

    # unpack every 2 bytes, little endian (16 bit encoding)
    unpack_symbols = ''.join([char*int(len(arr)/2) for char in 'h'])
    byte_array = struct.unpack(unpack_symbols,  arr)
    return np.array(byte_array)[::dwnsmpl]



def xml_to_np_array_file(path_to_xml, path_to_output = os.getcwd()):

    with open(path_to_xml, 'rb') as fd:
        dic = xmltodict.parse(fd.read().decode('utf8'))

    """

    Upload the ECG as numpy array with shape=[2500,12,1] ([time, leads, 1]).

    The voltage unit should be in 1 mv/unit and the sampling rate should be 250/second (total 10 second).

    The leads should be ordered as follow I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5, V6.

    """
    try:
        pt_id = dic['RestingECG']['PatientDemographics']['PatientID']
    except:
        print("no PatientID")
        pt_id = "none"
    try:
        PharmaUniqueECGID = dic['RestingECG']['PharmaData']['PharmaUniqueECGID']
    except:
        print("no PharmaUniqueECGID")
        PharmaUniqueECGID = "none"
    try:
        AcquisitionDateTime = dic['RestingECG']['TestDemographics']['AcquisitionDate'] + "_" + dic['RestingECG']['TestDemographics']['AcquisitionTime'].replace(":","-")
    except:
        print("no AcquisitionDateTime")
        AcquisitionDateTime = "none"    
        

    # try:
    #     requisition_number = dic['RestingECG']['Order']['RequisitionNumber']
    # except:
    #     print("no requisition_number")
    #     requisition_number = "none"

    #need to instantiate leads in the proper order for the model
    lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

    """
    Each EKG will have this data structure:
    lead_data = {
        'I': np.array
    }
    """

    lead_data =  dict.fromkeys(lead_order)
    #lead_data = {leadid: None for k in lead_order}

#     for all_lead_data in dic['RestingECG']['Waveform']:
#         for single_lead_data in lead['LeadData']:
#             leadname =  single_lead_data['LeadID']
#             if leadname in (lead_order):

    for lead in dic['RestingECG']['Waveform']:
        for leadid in range(len(lead['LeadData'])):
                sample_length = len(decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData']))
                #sample_length is equivalent to dic['RestingECG']['Waveform']['LeadData']['LeadSampleCountTotal']
                if sample_length == 5000:
                    lead_data[lead['LeadData'][leadid]['LeadID']] = decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData'], downsample = 0.5)
                elif sample_length == 2500:
                    lead_data[lead['LeadData'][leadid]['LeadID']] = decode_ekg_muse_to_array(lead['LeadData'][leadid]['WaveFormData'], downsample = 1)
                else:
                    continue
            #ensures all leads have 2500 samples and also passes over the 3 second waveform

    lead_data['III'] = (np.array(lead_data["II"]) - np.array(lead_data["I"]))
    lead_data['aVR'] = -(np.array(lead_data["I"]) + np.array(lead_data["II"]))/2
    lead_data['aVF'] = (np.array(lead_data["II"]) - np.array(lead_data["I"]))/2
    lead_data['aVL'] = (np.array(lead_data["I"]) - np.array(lead_data["II"]))/2
    
    lead_data = {k: lead_data[k] for k in lead_order}
    # drops V3R, V4R, and V7 if it was a 15-lead ECG

    # now construct and reshape the array
    # converting the dictionary to an np.array
    temp = []
    for key,value in lead_data.items():
        temp.append(value)

    #transpose to be [time, leads, ]
    ekg_array = np.array(temp).T

    #expand dims to [time, leads, 1]
    ekg_array = np.expand_dims(ekg_array,  axis=-1)

    # Here is a check to make sure all the model inputs are the right shape
#     assert ekg_array.shape == (2500, 12, 1), "ekg_array is shape {} not (2500, 12, 1)".format(ekg_array.shape )

    # filename = '/ekg_waveform_{}_{}.npy'.format(pt_id, requisition_number)
    filename = '{}_{}_{}.npy'.format(pt_id, AcquisitionDateTime,PharmaUniqueECGID)

    path_to_output += filename
    # print(path_to_output)
    with open(path_to_output, 'wb') as f:
        np.save(f, ekg_array)
        

def ekg_batch_run(ekg_list):
    i = 0
    x = 0
    for file in ekg_list:
        try:
            xml_to_np_array_file(file, output_dir)
            i+=1
        except Exception as e:
            # print("file failed: ", file)
            print(file, e)
            x+=1
        if i % 10000 == 0:
            print(f"Succesfully converted {i} EKGs, failed converting {x} EKGs")

            
# output_dir = os.getcwd() + '/ekg_waveforms_output/'
# print("args", sys.argv)
# ekg_file_list = []
# file_path(sys.argv[1])  #if you want input to be a directory
# print("Number of EKGs found: ", len(ekg_file_list))

# ekg_batch_run(ekg_file_list)


# To reconstruct the 12 lead ecg from the array
# test1 = np.load('waveform_output_example.npy')
# lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
# plt.rcParams["figure.figsize"] = [16,9]
# fig, axs = plt.subplots(len(lead_data))
# for i in range(0,12):
#     axs[i].plot(test1[:,i])
#     axs[i].set(ylabel=str(lead_order[i]))


# To find paced EKGs will use below, but work in progress
# dx_txt = [] 
# for line in dic['RestingECG']['Diagnosis']['DiagnosisStatement']:
#     dx_txt.append(line['StmtText'])
# print(dx_txt)


# Above script run in terminal

python3 /Users/pae2/HeartFailure/Test_waveforms/muse_xml_to_array.py ./DavidXML  <br />
args ['/Users/pae2/HeartFailure/Test_waveforms/muse_xml_to_array.py', './DavidXML']  <br />
Number of EKGs found:  1

In [15]:
all_arrays_train = np.array(all_arrays_train)

In [16]:
all_arrays_train.shape

(33, 1, 2500, 12)

In [17]:
reshaped = np.reshape(all_arrays_train,(all_arrays_train.shape[0],2500,12))

In [19]:
reshaped[0].shape

(2500, 12)

In [21]:

#To reconstruct the 12 lead ecg from the array
# test1 = np.load('/Users/pae2/Downloads/ekg_waveforms_output/201700237_09-22-2020_09-03-32_SCD11041674PA24092019090332.npy')
lead_order = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
plt.rcParams["figure.figsize"] = [16,9]
fig, axs = plt.subplots(len(lead_order))
for i in range(0,12):
    axs[i].plot(reshaped[0][:,i])
    axs[i].set(ylabel=str(lead_order[i]))

In [ ]:
# this is ecg_pdf_to_dataframe.py

# This is code we have written to create arrays from MUSE PDFs. It takes the raw vectors, converts to CVG, then csv arrays 

# # -*- coding: utf-8 -*-
# """
# Created on Fri Apr  6 13:44:59 2018
# Updated: Fri Oct 5 9:00 2018
# Updated: Tue Dec 10 2019
# Updated: Sat June 13 2020 (by Vijay Rajaram and Pierre Elias)
# @author: Sushravya Raghunath, David vanMaanen
# This step is processing traces in ECG PDFs into a dataframe.
# The final dataframe will have the following columns:
# ['PT_MRN','TEST_ID','filename','lead','signal','y_calibration','x_spacing']
# PT_MRN       : MRN of patient
# TEST_ID      : TestID of the ECG
# filename     : ECG PDF filename
# lead         : leadname of the traces in the row
# signal       : resampled signal
# y_calibration: the calibration factor applied while resampling
# x_spacing    : minimum time resolution between samples
# Notes:
# - calibration blip: short signal at the beginning of the signal traces for the leads.
# - If scale_x or scale_y is 0, likely the calibration blip shape was not typical.
# - The signal values correspond to the x and y positions from the PDF. This is subtracted to create a 0 baseline for the signal.
# """
 
# import asyncio
# from tempfile import mkstemp
# import os
# import subprocess
# from xml.dom.minidom import parse
# import numpy as np
# import pandas as pd
# import json
# from scipy.interpolate import interp1d
# from tqdm import tqdm
# from multiprocessing import Pool
# import sys
# from glob import glob
 
# def find_pdf_files(input_dir):
#   exts = ('pdf', 'PDF') # the tuple of file types
#   files = []
#   for ext in exts:
#     files.extend(glob(os.path.join(input_dir, "*.{}".format(ext))))
#   return files
 
# def resample_core(signalx, signaly, sqlen, minspacing):
#     '''
#     Input:
#         signalx : the x-axis data points
#         signaly : the signal at signalx points
#         sqlen   : The number of points the resampled signal
#         minspacing: The minimum spacing between two points
#     Output:
#         newy    : The resampled signal of sqlen # of points with spacing of minspacing
#     This function resamples the signal for the given minspacing to the given number of sample points.
#     '''
#     f = interp1d(signalx, signaly)
#     minnum = len(np.arange(signalx[0], signalx[-1], minspacing))
#     if minnum > sqlen:
#         raise Exception ('Minnum (', minnum,') greater than num (',sqlen,')')
#     xx = np.linspace(signalx[0], signalx[-1], sqlen)
#     newy = f(xx)
#     return(newy)
 
# async def convert_pdf_to_svg(fname, outname) -> int:
#     '''
#     Input:
#          fname   : PDF file name
#          outname : SVG file name
#     Output:
#          outname : return outname (file saved to disk)
#     This will convert PDF into SVG format and save it in the given outpath.
#     '''
#     cmd = f'pdftocairo -svg "{fname}" "{outname}"'
#     proc = await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
#     print("Started: %s, pid=%s" % (cmd, proc.pid), flush=True)
#     stdout, stderr = await proc.communicate()
#     if proc.returncode != 0:
#         if stdout:
#             print(stdout.decode())
#         if stderr:
#             print(f'Error in converting {fname} to SVG:\n{stderr.decode()}')
#     return proc.returncode
 
# def process_svg_to_pd_perdata(svgfile, pdffile=None):
#     '''
#     Input:
#     svgfile - datapath for svg file
#     Output (returns):
#     data    : data for 12 leads(available 15 or 12 traces), scale_vales and resolution units in a pandas dataframe
#     Hard coded values :
#     1) length of signal = 6 is assumed to be the calibration tracing at the beginning of the
#     trace (by experiment)
#     '''
#     columnnames = np.array(['I', 'II','III','aVR','aVL','aVF','V1','V2','V3','V4', \
#                      'V5', 'V6', 'V1L','IIL','V5L'])
#     doc = parse(svgfile)
#     if pdffile is None:
#         strn = 'FILENAME_NA'
#     else:
#         strn = os.path.splitext(os.path.basename(pdffile))[0]
#     data = pd.DataFrame(columns = ['PT_MRN','TEST_ID','filename','lead','x','y']) #,'scale_x','scale_y'])
#     a = 0
#     spacingvals = []
#     scale_vals = []
#     try:
#         siglen = []
#         for path in doc.getElementsByTagName('path'):
#             tmp = path.getAttribute('d')
#             tmp_split = tmp.split(' ')
#             signal_np = np.asarray([float(x) for x in tmp_split if (x != 'M' and x != 'L' and x != 'C' and x != 'Z' and x != '')])
#             signalx = signal_np[0::2]
#             signaly = signal_np[1::2]
#             siglen.append(len(signalx))
#         siglen = np.array(siglen)
#         # these are the calibration signals
#         cali6sigs = np.where(siglen == 6)[0]
#         minposcali = np.min(cali6sigs)
#         tmpstart = list(range(minposcali, len(siglen)))
#         last15sigs = np.array(list(set(tmpstart)- set(cali6sigs)))
#         # index for leads
#         a = 0
#         for ind, path in enumerate(doc.getElementsByTagName('path')):
#             if ind in last15sigs:
#                 if a > 14:
#                     continue
#                 tmp = path.getAttribute('d')
#                 tmp_split = tmp.split(' ')
#                 signal_np = np.asarray([float(x) for x in tmp_split if (x != 'M' and x != 'L' and x != 'C' and x != 'Z' and x != '')])
#                 signalx = signal_np[0::2]
#                 signaly = signal_np[1::2]
#                 # expect the name of the file to be ptmrn_testid format.
#                 tmp = strn.split('_')
#                 try:
#                     pid, testid = tmp[0], tmp[1]
#                 except:
#                     pid = tmp[0]
#                     testid = tmp[0]
#                 data.loc[data.shape[0]] = [pid, testid, strn, columnnames[a],signalx,signaly]
#                 spacingx = [t -s for s,t in zip(signalx, signalx[1:])]
#                 spacingvals.append(np.min(spacingx))
#                 a += 1
#             elif ind in cali6sigs:
#                 tmp = path.getAttribute('d')
#                 tmp_split = tmp.split(' ')
#                 signal_np = np.asarray([float(x) for x in tmp_split if (x != 'M' and x != 'L' and x != 'C' and x != 'Z' and x != '')])
#                 signalx = signal_np[0::2]
#                 signaly = signal_np[1::2]
#                 scale_vals.append([np.min(signaly), np.max(signaly)])
#         if len(scale_vals) == 0:
#             data = None
#             return data
#         sx = [x[0] for x in scale_vals]
#         sy = [x[1] for x in scale_vals]
#         startloc = [d[0] for d in data.x.values]
#         leads_ip = len(startloc)
#         a = np.sum(startloc[0:3] == startloc[0])
#         b = np.sum(startloc[3:6] == startloc[3])
#         c = np.sum(startloc[6:9] == startloc[6])
#         d = np.sum(startloc[9:12] == startloc[9])
#         if data.shape[0] == 15:
#             e = np.sum(startloc[12:15] == startloc[12])
#             checkrhs = [3,3,3,3,3]
#             checklhs = [a,b,c,d,e]
#             assert checklhs == checkrhs
#             scale_x = [sx[0:3],sx[0:3],sx[0:3],sx[0:3], sx[3:6]]
#             scale_y = [sy[0:3],sy[0:3],sy[0:3],sy[0:3], sy[3:6]]
#         elif data.shape[0] == 12:
#             checkrhs = [3,3,3,3]
#             checklhs = [a,b,c,d]
#             assert checklhs == checkrhs
#             scale_x = [sx[0:3],sx[0:3],sx[0:3],sx[0:3]]
#             scale_y = [sy[0:3],sy[0:3],sy[0:3],sy[0:3]]
#         else:
#             data=None
#             return data
#         scale_x = [y for x in scale_x for y in x]
#         data['scale_x'] = scale_x[0:data.shape[0]]
#         scale_y = [y for x in scale_y for y in x]
#         data['scale_y'] = scale_y[0:data.shape[0]]
#         data['minspacing'] = spacingvals[0:data.shape[0]]
#     except:
#         data =  None
#     return data
 
# def process_resample_data(data):
#     # This is a hard-coded specs for the data which is defined based on the signal sampling frequency and arrangement of the signals in the
#     # PDF file and the extraction mechanism used in `process_svg_to_pd_perdata`
#     config = {}
#     config['minspacing'] = 5.0
#     config['seqlen'] = [1250, 1250, 1250, 1250, 1250, 1250, 1250, 1250, 1250, 1250, 1250, 1250, 5000, 5000, 5000]
#     config['leadnames'] = ['I','II','III','aVR','aVL','aVF','V1','V2','V3','V4','V5','V6','V1L','IIL','V5L']
#     leadnames = config['leadnames'][0:data.shape[0]]
#     resampled_data = pd.DataFrame(columns = ['PT_MRN','TEST_ID','filename','lead','signal','y_calibration','x_spacing'])
#     for lead_iter in range(len(leadnames)):
#         lead_data = data.loc[data.lead == leadnames[lead_iter]]
#         #print(lead_data)
#         sqlen = config['seqlen'][lead_iter]
#         signalx = lead_data.x.values[0]
#         signaly = lead_data.y.values[0] - lead_data.scale_x.values[0]
#         calibration_y = (lead_data.scale_y.values[0] - lead_data.scale_x.values[0])/1000
#         signaly = signaly/calibration_y
#         newy = resample_core(signalx, signaly, sqlen, config['minspacing'])
#         newy = newy.astype(np.float32)
#         resampled_data.loc[resampled_data.shape[0]] = [lead_data.PT_MRN.values[0],
#                                                        lead_data.TEST_ID.values[0],
#                                                        lead_data.filename.values[0],
#                                                        leadnames[lead_iter],
#                                                        newy, calibration_y,
#                                                        lead_data.minspacing.values[0].round()]
#     return resampled_data
 
# async def get_data_from_PDF(pdf_path):
#     '''
#     Input:
#          pdf_path : path to the pdf
#     Output:
#          resampled_data : The final datafrane which contains resampled signal with other data specs.
#     '''
#     try:
#         if not os.path.exists('svgs'):
#             os.makedirs('svgs')
#         fp, svgfile = mkstemp(suffix='.svg', dir='svgs')
#         os.close(fp)
#         convertExitStatus = await convert_pdf_to_svg(pdf_path, svgfile)
#         assert 0 == convertExitStatus, f'convert_pdf_to_svg failed with status {convertExitStatus}'
#         data  = process_svg_to_pd_perdata(svgfile, pdf_path)
#         # Make sure all the signals have same sampling rate - it should be unless I picked up wrong signal from SVG and called it a signal
#         assert (data.minspacing.round() == data.minspacing[0].round()).all(), 'Sampling is different for different leads'
#         resampled_data = process_resample_data(data)
#         assert ((resampled_data.shape[0] == 12) | (resampled_data.shape[0] == 15)) == True, 'Number of signals extracted does not match 12 or 15'
#         return resampled_data
#     finally:
#         if os.path.exists(svgfile):
#             os.remove(svgfile)
 
# def create_dataframe(pdf_path):
#   try:
#       if sys.version_info >= (3, 7):
#           finaldataframe = asyncio.run(get_data_from_PDF(pdf_path))
#       else:
#           loop = asyncio.get_event_loop()
#           finaldataframe = loop.run_until_complete(get_data_from_PDF(pdf_path))
#       return finaldataframe
#   except Exception as e:
#       print('Error in PDF : {} with error {}'.format(pdf_path, e))
 
# def expand_list(df, list_column, new_column): 
#     lens_of_lists = df[list_column].apply(len)
#     origin_rows = range(df.shape[0])
#     destination_rows = np.repeat(origin_rows, lens_of_lists)
#     non_list_cols = (
#       [idx for idx, col in enumerate(df.columns)
#        if col != list_column]
#     )
#     expanded_df = df.iloc[destination_rows, non_list_cols].copy()
#     expanded_df[new_column] = (
#       [item for items in df[list_column] for item in items]
#       )
#     expanded_df.reset_index(inplace=True, drop=True)
#     return expanded_df
 
# if __name__ == '__main__':
#     current_dir = os.getcwd()
#     print("args", sys.argv)
#     # pdf_paths = [os.path.join(current_dir, sys.argv[1])] #if you want input to be a file
#     pdf_paths = find_pdf_files(sys.argv[1]) #if you want input to be a directory
#     print(pdf_paths)
#     # PARALLEL
#     with Pool(processes=16) as pool: #change processes based off CPU thread count 
#         # use imap instead of imap_unordered to retain order (easier for printing purposes)
#         dfs = [df for df in tqdm(pool.imap(create_dataframe, pdf_paths), total = len(pdf_paths))]
#         for pdf_path, df in zip(pdf_paths, dfs):
#           # print(pdf_path, "\n", df, "\n\n") #if you want to print each individual ecg df 
#           filename_csv=str(df['filename'][0])+'.csv'
#           filename_pkl = str(df['filename'][0])+'.pkl'
#           # df.to_pickle(filename_pkl)
#           expanded_df = expand_list(df, "signal", "signal") #this explodes the list in df['signal'], creating 30k rows per ECG
#           expanded_df.to_csv(filename_csv, index=False)
#           df['signal'] = str(df['signal'])
#           df.to_csv('test_df.csv', index=False)
 
#     # OR SEQUENTIAL
#     # for pdf_path in tqdm(pdf_paths, total = len(pdf_paths)):
#     #     df = create_dataframe(pdf_path)
#     #     print(pdf_path, "\n", df, "\n\n")
 
 
